**2025/03/25**

This script processes leveling benchmark data and vertical displacement time-series.
It performs the following steps:
1. Load a leveling benchmark shapefile and vertical displacement time-series.
2. Extract coordinate information (X_TWD97 and Y_TWD97) from a provided geospatial info file.
3. Map these coordinates to the time-series DataFrame and convert it into a GeoDataFrame.
4. For each leveling benchmark, find neighboring InSAR points within a defined buffer radius.
5. Combine the neighbor information into a single GeoDataFrame and save the result as a pickle file.

This allows you to correlate InSAR displacement data with leveling benchmark locations.

```{python}
from my_packages import *
from appgeopy import *

# File paths for input data
LEVELING_SHP_PATH = r"E:\SUBSIDENCE_PROJECT_DATA\地陷資料整理\水準點\LevelingBenchmarks_DavidNCU\shapefiles\leveling_benchmark_location.shp"
HDF5_FILE_PATH = r"D:\1000_SCRIPTS\003_Project002\20241219_LevelingData_to_HDF5\20241220_All_LevelingData_LandSubsidence-wra-gov-tw.h5"

# Load the leveling benchmark shapefile
print("Loading leveling benchmark shapefile...")
leveling_shp = gpd.read_file(LEVELING_SHP_PATH)

# Load the geospatial information (geometry and angles)
info_gdf = pd.read_pickle("ASC_DESC_inc_azim.xz")

# Extract X and Y coordinates from the geometry column and insert them as new columns
x_twd97_arr = [ele.x for ele in info_gdf['geometry']]
y_twd97_arr = [ele.y for ele in info_gdf['geometry']]
info_gdf.insert(loc=0, column="Y_TWD97", value=y_twd97_arr)
info_gdf.insert(loc=0, column="X_TWD97", value=x_twd97_arr)

# Load the monthly resampled vertical cumulative displacement time-series
monthly_resampled_vertical_cumdisp = pd.read_pickle(r"MonthlyResample_dU_timeseries.pkl", compression='zip')

# Map X_TWD97 and Y_TWD97 from the geospatial info DataFrame to the time-series DataFrame using the index (assumed to be point identifiers)
monthly_resampled_vertical_cumdisp["X_TWD97"] = monthly_resampled_vertical_cumdisp.index.map(info_gdf["X_TWD97"])
monthly_resampled_vertical_cumdisp["Y_TWD97"] = monthly_resampled_vertical_cumdisp.index.map(info_gdf["Y_TWD97"])

# Convert the displacement DataFrame into a GeoDataFrame using the coordinates and the specified CRS
monthly_resampled_vertical_cumdisp = geospatial.convert_to_geodata(
    monthly_resampled_vertical_cumdisp,
    xcoord_col="X_TWD97",
    ycoord_col="Y_TWD97",
    crs_epsg="EPSG:3826"
)

# Check the type to confirm it is a GeoDataFrame
print("Type of monthly_resampled_vertical_cumdisp:", type(monthly_resampled_vertical_cumdisp))  # should be GeoDataFrame

# Define the key column used as the benchmark identifier and buffer search radius (in meters)
CENTRAL_KEY_COLUMN = "樁號"
BUFFER_RADIUS = 500  # Buffer radius for InSAR point search, in meters

# Find neighbor points for each leveling benchmark
print("Finding neighbors for each leveling benchmark...")
results = []  # to store neighbor search results for each benchmark

# Iterate over each benchmark using tqdm for progress tracking
for idx, row in tqdm(leveling_shp.iterrows(), total=len(leveling_shp)):
    try:
        # Use a helper function to find InSAR point neighbors within the buffer radius
        result = geospatial.find_point_neighbors(row, monthly_resampled_vertical_cumdisp, CENTRAL_KEY_COLUMN, BUFFER_RADIUS)
        results.append(result)
    except Exception as e:
        print(f"Error processing row {idx}: {e}")

# Combine all individual results into a single GeoDataFrame
result_gdf = pd.concat(results, ignore_index=True)

# Save the combined results to a pickle file with a descriptive filename
save_path = f"MonthlyResample_dU_timeseries_add_Benchmarks_{BUFFER_RADIUS}m.pkl"
result_gdf.to_pickle(save_path)

print("Processing complete. Results saved to:", save_path)
```

**2025/03/25**

```{python}
"""
This script compares leveling measurements with InSAR vertical displacement time-series 
for benchmarks within a specified buffer zone.

Workflow:
1. Load the InSAR vertical displacement GeoDataFrame (which has benchmark neighbors).
2. Extract the buffer radius from the filename.
3. Load leveling measurement and metadata from an HDF5 file.
4. Identify unique benchmark IDs (using the key column "樁號") in the buffer zone.
5. Filter benchmarks to keep only those available in the HDF5 database.
6. Further filter benchmarks to only include those that have data covering a given time interval.
7. For each valid benchmark:
   a. Extract and process the leveling data (calculate leveling velocity in mm/yr).
   b. Extract the corresponding InSAR data and compute a velocity estimate (average and standard deviation).
   c. Store the results.
8. Save the comparison results to an Excel file.
"""

# --- Import required libraries and helper functions ---
from appgeopy import *
from my_packages import *

file2process = glob("MonthlyResample*m.pkl")

for fpath in file2process:
    print(f"\n\nCurrently process: {fpath}")
    # --- Define file paths and parameters ---
    # fpath = r"MonthlyResample_dU_timeseries_add_Benchmarks_500m.pkl"
        
    # Extract buffer radius from the filename (assumes the radius is the last underscore-separated token before the extension)
    BUFFER_RADIUS = fpath.split(".")[0].split("_")[-1]
    
    # Load the InSAR displacement GeoDataFrame (which includes benchmark neighbor information)
    result_gdf = pd.read_pickle(fpath)
    CENTRAL_KEY_COLUMN = "樁號"  # Column used to identify benchmarks
    
    # Leveling data HDF5 file path (contains measurements and metadata)
    HDF5_FILE_PATH = r"D:\1000_SCRIPTS\003_Project002\20241219_LevelingData_to_HDF5\20241220_All_LevelingData_LandSubsidence-wra-gov-tw.h5"
    
    # --- Open the HDF5 file and extract leveling data dictionaries ---
    leveling_measure_dict, leveling_metadata_dict = open_HDF5(HDF5_FILE_PATH)
    
    # --- Identify unique benchmarks present in the InSAR buffer zone ---
    unique_leveling_StationNumber = result_gdf[CENTRAL_KEY_COLUMN].unique()
    
    # Determine benchmarks that are also available in the leveling database
    valid_leveling_StationNumber = sorted(
        set(leveling_metadata_dict.keys()).intersection(set(unique_leveling_StationNumber))
    )
    
    # --- Filter Benchmarks by Time Interval ---
    START_YEAR = 2016
    END_YEAR = 2023
    
    satisfied_leveling_StationNumber = []
    print("Filtering benchmarks by time interval...")
    for select_StationNumber in tqdm(valid_leveling_StationNumber):
        metadata = leveling_metadata_dict[select_StationNumber]
        # Convert record dates to datetime
        first_record_date = pd.to_datetime(metadata["First_Record"])
        last_record_date = pd.to_datetime(metadata["Last_Record"])
        # Check if the benchmark covers the desired time interval
        if first_record_date.year <= START_YEAR and last_record_date.year >= END_YEAR:
            satisfied_leveling_StationNumber.append(select_StationNumber)
    
    # --- Compare Leveling and InSAR Measurements ---
    # Initialize a cache dictionary to store the results for each benchmark.
    cache = {"StationNumber": [], "Leveling_mm_yr": [], "InSAR_mm_yr": [], "InSAR_stdev": []}
    
    print("Comparing leveling and InSAR measurements...")
    for select_StationNumber in tqdm(satisfied_leveling_StationNumber):
        try:
            # --- Process Leveling Measurements ---
            # Extract leveling data for the current benchmark and decode the 'date' column
            leveling_data = pd.DataFrame(leveling_measure_dict[select_StationNumber])
            leveling_data["date"] = leveling_data["date"].apply(lambda x: x.decode("utf-8"))
            leveling_data["date"] = pd.to_datetime(leveling_data["date"])
            leveling_data.set_index("date", inplace=True)
            # Subset the leveling data for the specified time interval
            subset_leveling_data = leveling_data.loc[str(START_YEAR) : str(END_YEAR)]
            
            # Calculate leveling velocity using a polynomial trend (order 1 = linear trend)
            fulltime_subset_leveling_data = dataframe_handle.convert_to_fulltime(subset_leveling_data)
            trend, coeffs = analysis.get_polynomial_trend(series=fulltime_subset_leveling_data["values"], order=1)
            leveling_slope = coeffs[-1] * 365.25  # Convert to m/year
            leveling_slope_mm = leveling_slope * 1000  # Convert to mm/year
            
            # --- Process InSAR Measurements ---
            # Filter InSAR data for the current benchmark
            insar_data = result_gdf.query(f"{CENTRAL_KEY_COLUMN}==@select_StationNumber")
            measure_cols = [col for col in insar_data.columns if col.startswith("D")]
            insar_data = insar_data.loc[:, measure_cols]
            # Convert column names (date strings with a leading letter) into datetime objects
            insar_data.columns = pd.to_datetime([col[1:] for col in insar_data.columns])
            # Convert InSAR data to a full time series (if required by the analysis function)
            insar_data = dataframe_handle.convert_to_fulltime(insar_data.T)
            
            # Function to calculate InSAR velocity from a time-series (slope in m/day converted to mm/year)
            def get_insar_velocity(series):
                trend, slope = analysis.get_linear_trend(series)
                return slope * 365.25
            
            # Apply the linear trend function to each column to obtain point velocities
            insar_point_velocities = insar_data.apply(get_insar_velocity, axis=0)
            insar_point_velocities_average = insar_point_velocities.mean()
            insar_point_velocities_stdev = insar_point_velocities.std()
            
            # --- Save Results to Cache ---
            cache["StationNumber"].append(select_StationNumber)
            cache["Leveling_mm_yr"].append(leveling_slope_mm)
            cache["InSAR_mm_yr"].append(insar_point_velocities_average)
            cache["InSAR_stdev"].append(insar_point_velocities_stdev)
        
        except Exception as e:
            print(f"Error processing {select_StationNumber}: {e}")
            pass
    
    # --- Save Results ---
    # Define output file path for the Excel file containing comparison results.
    output_path = os.path.join("monthly_dU_validation", f"Leveling_InSAR_inBuffer_{BUFFER_RADIUS}.xlsx")
    # Convert the cache dictionary into a DataFrame and save to Excel.
    leveling_insar_df = pd.DataFrame(cache)
    leveling_insar_df.to_excel(output_path, index=False)
    
    print(f"Results saved to {output_path}")
```

**2025/03/26**

```{python}
"""
This script creates density scatter plots comparing InSAR and leveling 
measurements (in mm/year) and exports the plots as PNG images. 

Workflow:
1. Define a function to create a density scatter plot based on 2D histogram interpolation.
2. Define a function to set up the figure, add labels, reference lines, and statistical annotations.
3. Loop through Excel files (one per benchmark) in the specified folder, filter outliers,
   and export the resulting scatter plot.
"""

# --- Import necessary libraries ---
from my_packages import *
from appgeopy import *

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import Normalize
import matplotlib.ticker as plticker
from scipy.interpolate import interpn
from glob import glob
from tqdm import tqdm

# --- Function Definitions ---

def density_scatter(x, y, ax=None, sort=True, bins=20, **kwargs):
    """
    Create a density scatter plot.
    
    This function calculates a 2D histogram for the (x, y) data, 
    interpolates density values at the scatter points, optionally sorts 
    points by density (to plot densest points last), and returns the Axes.
    
    Parameters:
        x (array-like): x-values.
        y (array-like): y-values.
        ax (matplotlib.axes.Axes, optional): Axis to plot on; creates new figure if None.
        sort (bool): If True, sort points by density.
        bins (int): Number of bins to use for the 2D histogram.
        **kwargs: Additional keyword arguments (unused).
    
    Returns:
        ax: The matplotlib Axes with the density scatter plot.
    """
    if ax is None:
        fig, ax = plt.subplots()

    # Calculate 2D histogram for density estimation
    data, x_e, y_e = np.histogram2d(x, y, bins=bins, density=False)

    # Interpolate density values for each scatter point
    z = interpn(
        (0.5 * (x_e[1:] + x_e[:-1]), 0.5 * (y_e[1:] + y_e[:-1])),
        data,
        np.vstack([x, y]).T,
        method="splinef2d",
        bounds_error=False,
    )

    # Replace NaN values in density with zero
    z[np.isnan(z)] = 0.0

    # Optionally sort points by density (dense points plotted on top)
    if sort:
        idx = z.argsort()
        x, y, z = x[idx], y[idx], z[idx]

    # Create scatter plot with color mapping based on density
    scatter = ax.scatter(x, y, c=z, cmap="turbo", s=200, edgecolor="none", linewidths=0.75)

    # Add color bar with appropriate tick intervals
    norm = Normalize(vmin=np.min(z), vmax=np.max(z))
    cbar = ax.get_figure().colorbar(ScalarMappable(norm=norm, cmap="turbo"), ax=ax, shrink=0.4, pad=0.01)
    cbar.ax.set_ylabel("Points / Unit Area", fontsize=16)
    cbar.ax.tick_params(labelsize=14)
    cbar.ax.yaxis.set_major_locator(MaxNLocator(integer=True, prune="lower"))
    
    return ax


def export_scatterplot(datatable, folder2save, savename, saveFig=True, showFig=False):
    """
    Create and export a density scatter plot comparing InSAR and leveling velocities.
    
    The function sets up a square figure, plots a dashed reference line, 
    overlays a density scatter plot of the InSAR vs. leveling velocities, and 
    annotates the plot with Mean Absolute Error (MAE), RMSE, and correlation (r).
    Finally, the plot is saved to the specified folder.
    
    Parameters:
        datatable (pd.DataFrame): DataFrame containing the comparison data.
        folder2save (str): Folder path to save the figure.
        savename (str): Filename for the saved figure.
        saveFig (bool): If True, the figure is saved.
        showFig (bool): If True, the figure is displayed.
    """
    # Figure settings
    multiplier = 1.5
    fig = plt.figure(figsize=(10 * multiplier, 10 * multiplier))
    ax = fig.add_subplot(1, 1, 1)
    ax.grid(axis="both", which="major", color="lightgrey", alpha=0.5)
    
    # Plot reference dashed line (y = x)
    top_thres = 2
    bot_thres = -8
    ax.plot([top_thres, bot_thres], [top_thres, bot_thres], linestyle="--", color="gray", linewidth=4, zorder=1)
    ax.set_aspect(aspect="equal", adjustable="box")
    
    # Define axis objects and labels
    x_axis_object = "InSAR_mm_yr"
    y_axis_object = "Leveling_mm_yr"
    x_axis_label = r"${\nu}_{InSAR}$ (mm/year)"
    y_axis_label = r"${\nu}_{leveling}$ (mm/year)"
    
    # Extract x and y data for the scatter plot
    x = datatable[x_axis_object]
    y = datatable[y_axis_object]
    
    # Create the density scatter plot
    density_scatter(x, y, ax=ax, bins=50)
    
    # Set axis labels with font sizes and label padding
    ax.set_xlabel(x_axis_label, fontsize=45, labelpad=15)
    ax.set_ylabel(y_axis_label, fontsize=45, labelpad=15)
    
    # Compute statistical metrics for annotations
    mean_abs_err = np.mean(np.abs(x - y))
    rmse = np.sqrt(np.mean((x - y) ** 2))
    m, c, r, p, se1 = stats.linregress(x, y)
    
    # Set tick locators for both axes
    major_base = 10
    minor_base = 5
    ax.xaxis.set_major_locator(plticker.MultipleLocator(base=major_base))
    ax.xaxis.set_minor_locator(plticker.MultipleLocator(base=minor_base))
    ax.yaxis.set_major_locator(plticker.MultipleLocator(base=major_base))
    ax.yaxis.set_minor_locator(plticker.MultipleLocator(base=minor_base))
    
    # Set tick parameters for readability
    ax.tick_params(axis="both", which="major", labelsize=45, direction="out", length=16, width=2)
    ax.tick_params(axis="both", which="minor", labelsize=45, direction="out", length=12, width=2)
    
    # Add text annotations for MAE, RMSE, and correlation coefficient
    ax.text(0.075, 0.95, f"MAE = {mean_abs_err:.2f} mm/year", transform=ax.transAxes, fontweight="bold", fontsize=40)
    ax.text(0.075, 0.91, f"RMSE = {rmse:.2f} mm/year", transform=ax.transAxes, fontweight="bold", fontsize=40)
    ax.text(0.075, 0.87, f"r = {r:.2f}", transform=ax.transAxes, fontweight="bold", fontsize=40)
    
    # Remove top and right borders and add reference lines at zero
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.axhline(0, color="lightgrey")
    ax.axvline(0, color="lightgrey")
    
    fig.tight_layout()
    
    # Save or show the figure
    if saveFig:
        plt.savefig(os.path.join(folder2save, savename), dpi=600, bbox_inches="tight")
    if showFig:
        plt.show()
    plt.close()

# --- Main Processing Loop ---

# Define the folder containing Excel files to process
mainfolder = r"monthly_dU_validation"
file2process = glob(os.path.join(mainfolder, "Leveling_*.xlsx"))

# Loop through each Excel file and create the corresponding scatter plot
for filepath in file2process:
    # Derive the output image name from the Excel filename
    fig_savename = os.path.basename(filepath).replace(".xlsx", ".png")
    
    # Read the data from the Excel file
    df = pd.read_excel(filepath)
    
    # Compute squared differences between leveling and InSAR velocities
    df["SqrDiff"] = (df["Leveling_mm_yr"] - df["InSAR_mm_yr"]) ** 2
    
    # Filter out outliers using the 5th and 95th quantiles of SqrDiff
    lower_bound = np.quantile(df["SqrDiff"], 0.05)
    upper_bound = np.quantile(df["SqrDiff"], 0.95)
    df = df.query("SqrDiff >= @lower_bound & SqrDiff <= @upper_bound").reset_index(drop=True)
    
    # Export the scatter plot using the export_scatterplot function
    export_scatterplot(datatable=df, folder2save=mainfolder, savename=fig_savename)

```